# Normal Year vs. Multi-Year Drought

## Retail Comparison

Identifies which utilities experience a significant change in water supply and demand in normal years and multi-year droughts.

## Set-up

Import packages and data.

In [1]:
# Import packages
import numpy as np
import pandas as pd

In [2]:
# Import normal year retail excel table
normal = pd.read_csv('normal_retail.csv')

# Import multi-year drought retail excel table
drought = pd.read_csv('drought_retail.csv')

Create normal and drought supply & demand tables.

In [3]:
# Normal Supply Retail
normal_supply = normal[normal['TOTAL_TYPE'] == 'Supply totals']
normal_supply = normal_supply.drop(columns = ['ORG_ID', 'WORKSHEET_NAME', 'REVIEWED_BY_DWR', 'REQUIREMENTS_ADDRESSED', 'TOTAL_TYPE'])
normal_supply = normal_supply.rename(index = str, columns = {'WATER_SUPPLIER_NAME':'water_supplier', 'TOTAL_VOLUME_2020_AF':'2020', 'TOTAL_VOLUME_2025_AF':'2025', 'TOTAL_VOLUME_2030_AF':'2030', 'TOTAL_VOLUME_2035_AF':'2035', 'TOTAL_VOLUME_2040_AF':'2040'})

In [4]:
# Normal Demand Retail
normal_demand = normal[normal['TOTAL_TYPE'] == 'Demand totals']
normal_demand = normal_demand.drop(columns = ['ORG_ID', 'WORKSHEET_NAME', 'REVIEWED_BY_DWR', 'REQUIREMENTS_ADDRESSED', 'TOTAL_TYPE'])
normal_demand = normal_demand.rename(index = str, columns = {'WATER_SUPPLIER_NAME':'water_supplier', 'TOTAL_VOLUME_2020_AF':'2020', 'TOTAL_VOLUME_2025_AF':'2025', 'TOTAL_VOLUME_2030_AF':'2030', 'TOTAL_VOLUME_2035_AF':'2035', 'TOTAL_VOLUME_2040_AF':'2040'})

In [5]:
# Drought Supply Retail
drought_supply = drought[drought['TOTAL_TYPE'] == 'Supply totals']
drought_supply = drought_supply.drop(columns = ['ORG_ID', 'WORKSHEET_NAME', 'REVIEWED_BY_DWR', 'REQUIREMENTS_ADDRESSED', 'WP_WUEDATA_PLAN_ID', 'TOTAL_TYPE'])
drought_supply = drought_supply.rename(index = str, columns = {'WATER_SUPPLIER_NAME':'water_supplier', 'YEAR_TYPE':'year_type', 'TOTAL_VOLUME_2020_AF':'2020', 'TOTAL_VOLUME_2025_AF':'2025', 'TOTAL_VOLUME_2030_AF':'2030', 'TOTAL_VOLUME_2035_AF':'2035', 'TOTAL_VOLUME_2040_AF':'2040'})
drought_supply = drought_supply.dropna()

In [6]:
# Drought Demand Retail
drought_demand = drought[drought['TOTAL_TYPE'] == 'Demand totals']
drought_demand = drought_demand.drop(columns = ['ORG_ID', 'WORKSHEET_NAME', 'REVIEWED_BY_DWR', 'REQUIREMENTS_ADDRESSED', 'WP_WUEDATA_PLAN_ID', 'TOTAL_TYPE'])
drought_demand = drought_demand.rename(index = str, columns = {'WATER_SUPPLIER_NAME':'water_supplier', 'YEAR_TYPE':'year_type', 'TOTAL_VOLUME_2020_AF':'2020', 'TOTAL_VOLUME_2025_AF':'2025', 'TOTAL_VOLUME_2030_AF':'2030', 'TOTAL_VOLUME_2035_AF':'2035', 'TOTAL_VOLUME_2040_AF':'2040'})
drought_demand = drought_demand.dropna()

## 1) Supply

Find percent change in utilities' supplies.

In [7]:
# Select suppliers that are present in both normal & drought data
drt_sup = drought_supply.loc[drought_supply['water_supplier'].isin(normal_supply['water_supplier'])]
nrm_sup = normal_supply.loc[normal_supply['water_supplier'].isin(drt_sup['water_supplier'])]

In [8]:
# Normal & drought water suppliers
norm_sup = nrm_sup['water_supplier']
drt_sup = drt_sup['water_supplier']

# Check if suppliers are the same
# norm_sup.unique() == drt_sup.unique()

# Create supply change DataFrame
chg_sup = pd.DataFrame(columns = ['water_supplier', 'supply_change'])

Change in supplies for third year drought & normal year.

In [10]:
normal_supply.head()

,water_supplier,2020,2025,2030,2035,2040
1,Adelanto City Of,9300.0,10444.0,11581.0,12640.0,13750.0
3,Alameda County Water District,77200.0,76900.0,76600.0,76300.0,76000.0
5,Alhambra City Of,12913.0,13273.0,13634.0,13995.0,14354.0
6,Amador Water Agency,18343.0,18884.0,18884.0,18884.0,18884.0
8,American Canyon City Of,7560.0,8265.0,8470.0,8981.0,8981.0


In [12]:
norm_sup.head()

1                Adelanto  City Of
3    Alameda County Water District
5                Alhambra  City Of
6              Amador Water Agency
8         American Canyon  City Of
Name: water_supplier, dtype: object

In [9]:
# For each normal water supplier 
for n_index, n_utility in norm_sup.iteritems():
    
    # Create array of normal supplies - each column is a year (2020, 2025, etc.)
    nrm = normal_supply.loc[n_index, :][1:6].to_numpy()
    
    # For each drought water supplier
    for d_index, d_utility in drt_sup.iteritems():
        
        # If normal supplier == drought supplier
        if n_utility == d_utility:
            
            # Year type of drought (i.e. First year, Second year, Third Year)
            year_type = drought_supply['year_type'][d_index]
           
            # If Year type = Third year
            if 'Third' in year_type:
                
                # Convert drought supply for third year into array
                drt = drought_supply.loc[d_index, :][2:7].to_numpy()
        
                # Find difference between normal & drought supplies for third year
                differ = drt - nrm
                
                # If normal & drought supplies are not the same for all five years (2020, 2025, etc.)
                if all([difference != 0 for difference in differ]):
                    
                    # Find change between normal & drought supplies
                    change = differ / nrm
                    
                    # Add utility name & supply change to the supply change DataFrame
                    chg_sup = chg_sup.append({'water_supplier': d_utility, 'supply_change': change}, ignore_index = True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Average change in supplies for third year drought & normal year.

In [10]:
# Change supplies
chg_supply = chg_sup['supply_change']

# Create average change DataFrame
avg_chg_sup = pd.DataFrame(columns = ['water_supplier', 'supply_avg_chg', '+/-'])

In [11]:
# For each water supplier
for c_index, c_change in chg_supply.iteritems():
    
    # Determine utility name
    utility = chg_sup.loc[c_index, 'water_supplier']
    
    # For yearly supply change
    for i, change in enumerate(c_change):
        
        # If change is infite
        if change == np.inf :
            
            # Set change to zero
            c_change[i] = 0

    # Find average supply change for each supplier
    average = np.mean(c_change)
    
    # Determine if change is positive or negative (positive = supplies increase in drought, negative = supplies decrease in drought)
    if average > 0:
        sign = '+'
    else:
        sign = '-'
    
    # Add utility name & average supply change to the average change DataFrame
    avg_chg_sup = avg_chg_sup.append({'water_supplier': utility, 'supply_avg_chg': abs(average), '+/-': sign}, ignore_index = True)

Top 50 utilities in regards to supply changes.

In [12]:
# Top 50 absolute supply changes
top_50_sup = avg_chg_sup.nlargest(50, 'supply_avg_chg')

### Chowchilla Report Discrepancy?

Supplies drastically increase in drought years compared to normal years, by a whole order of magnitude. Possible reporting error.

In [13]:
normal_supply[normal_supply['water_supplier'] == 'Chowchilla, City of Water Department']

,water_supplier,2020,2025,2030,2035,2040
150,"Chowchilla, City of Water Department",5149.593812,6898.859887,8467.061578,10701.21193,11661.77383


In [14]:
drought_supply[drought_supply['water_supplier'] == 'Chowchilla, City of Water Department'][drought_supply['year_type'] == 'Third year']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,water_supplier,year_type,2020,2025,2030,2035,2040
907,"Chowchilla, City of Water Department",Third year,491807.6913,491807.6913,491807.6913,491807.6913,491807.6913


## 2) Demand

Find percent change in utilities' demands.

In [15]:
# Select suppliers that are present in both normal & drought data
drt_dem = drought_demand.loc[drought_demand['water_supplier'].isin(normal_supply['water_supplier'])]
nrm_dem = normal_demand.loc[normal_demand['water_supplier'].isin(drt_dem['water_supplier'])]

In [16]:
# Normal & drought water suppliers
norm_dem = nrm_dem['water_supplier']
drt_dem = drt_dem['water_supplier']

# Check if suppliers are the same
# norm_dem.unique() == drt_dem.unique()

# Create demand change DataFrame
chg_dem = pd.DataFrame(columns = ['water_supplier', 'demand_change'])

Change in demands for third year drought & normal year.

In [17]:
# For each normal water supplier 
for n_index, n_utility in norm_dem.iteritems():
    
    # Create array of normal demand - each column is a year (2020, 2025, etc.)
    nrm = normal_demand.loc[n_index, :][1:6].to_numpy()
    
    # For each drought water supplier
    for d_index, d_utility in drt_dem.iteritems():
        
        # If normal supplier == drought supplier
        if n_utility == d_utility:
            
            # Year type of drought (i.e. First year, Second year, Third Year)
            year_type = drought_demand['year_type'][d_index]

            # If Year type = Third year
            if 'Third' in year_type:
                
                # Convert drought supply for third year into array
                drt = drought_demand.loc[d_index, :][2:7].to_numpy()
        
                # Find difference between normal & drought demand for third year
                differ = drt - nrm
                
                # If normal & drought demand are not the same for all five years (2020, 2025, etc.)
                if all([difference != 0 for difference in differ]):
                    
                    # Find change between normal & drought demand
                    change = differ / nrm
                    
                    # Add utility name & demand change to the demand change DataFrame
                    chg_dem = chg_dem.append({'water_supplier': d_utility, 'demand_change': change}, ignore_index = True)

Average change in demands for third year drought & normal year.

In [18]:
# Change demands
chg_demand = chg_dem['demand_change']

# Create average change DataFrame
avg_chg_dem = pd.DataFrame(columns = ['water_supplier', 'demand_avg_chg', '+/-'])

In [19]:
# For each water supplier
for c_index, c_change in chg_demand.iteritems():
    
    # Determine utility name
    utility = chg_dem.loc[c_index, 'water_supplier']
    
    # For yearly supply change
    for i, change in enumerate(c_change):
        
        # If change is infite
        if change == np.inf :
            
            # Set change to zero
            c_change[i] = 0

    # Find average demand change for each supplier
    average = np.mean(c_change)
    
    # Determine if change is positive or negative (positive = demands increase in drought, negative = demands decrease in drought)
    if average > 0:
        sign = '+'
    else:
        sign = '-'
    
    # Add utility name & average demand change to the average change DataFrame
    avg_chg_dem = avg_chg_dem.append({'water_supplier': utility, 'demand_avg_chg': abs(average), '+/-': sign}, ignore_index = True)

Top 50 utilities in regards to supply changes.

In [20]:
# Top 50 absolute demand changes
top_50_dem = avg_chg_dem.nlargest(50, 'demand_avg_chg')

### South Feather Report Discrepancy?

Demands drastically increase in drought years compared to normal years, by a whole order of magnitude. Possible reporting error.

In [21]:
normal_demand[normal_demand['water_supplier'] == 'South Feather Water and Power']

,water_supplier,2020,2025,2030,2035,2040
681,South Feather Water and Power,4004.898644,4118.447495,4176.756364,4231.996345,4290.305214


In [22]:
drought_demand[drought_demand['water_supplier'] == 'South Feather Water and Power'][drought_demand['year_type'] == 'Third year']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,water_supplier,year_type,2020,2025,2030,2035,2040
4091,South Feather Water and Power,Third year,12290.89584,12637.68017,12818.74455,12987.53338,13165.52888


## 2.5) Top 50 Supply & Demand

Determine which utilities are in the top 50 for both supply and demand.

In [23]:
# Supply & demand water suppliers
sup_50 = top_50_sup['water_supplier']
dem_50 = top_50_dem['water_supplier']

# Create both top 50 dataframe
both_top_50 = pd.DataFrame(columns = ['water_supplier', 'supply_avg_chg', '+/-', 'demand_avg_chg', '+/-'])

In [24]:
# For each top 50 supplier 
for s_index, s_utility in sup_50.iteritems():
    
    # For each top 50 demander
    for d_index, d_utility in dem_50.iteritems():
        
        # If supplier == demander
        if s_utility == d_utility:
            
            # If utility not already listed in both_top_50
            if d_utility not in both_top_50['water_supplier']:
                
                # Find supply data
                sup_chg = top_50_sup['supply_avg_chg'][s_index]
                sup_sign = top_50_sup['+/-'][s_index]

                # Find demand data
                dem_chg = top_50_dem['demand_avg_chg'][d_index]
                dem_sign = top_50_dem['+/-'][d_index]

                # Add utility name, change & sign to the both top 50 DataFrame
                both_top_50 = both_top_50.append({'water_supplier': d_utility, 'supply_avg_chg': sup_chg, '+/-': sup_sign, 'demand_avg_chg': dem_chg, '+/-': dem_sign}, ignore_index = True)

## 3) Supply - Demand

Supply minus demand for third year drought, years 2020-2035.

In [25]:
# Third year drought supply
drt3_sup = drought_supply[drought_supply['year_type'] == 'Third year'].drop(columns = ['year_type', '2040'])

# Third year drought demand
drt3_dem = drought_demand[drought_demand['year_type'] == 'Third year'].drop(columns = ['year_type', '2040'])

In [26]:
# Third year drought water suppliers & demanders
drought3_sup = drt3_sup['water_supplier']
drought3_dem = drt3_dem['water_supplier']

# Create third year supply minus demand DataFrame
sup_minus_dem = pd.DataFrame(columns = ['water_supplier', '2020', '2025', '2030', '2035'])

In [27]:
# For each third year drought supplier
for s_index, s_utility in drought3_sup.iteritems():
    
    # For each third year drought demander
    for d_index, d_utility in drought3_dem.iteritems():
        
        # If supplier == demander
        if s_utility == d_utility:
            
            # Find supplier quantities
            sup_values = drt3_sup.loc[s_index][1:6].to_numpy()
            
            # Find demander quantities
            dem_values = drt3_dem.loc[d_index][1:6].to_numpy()
            
            # Find supply - demand
            dif = sup_values - dem_values
                    
            # Add utility name & supply - demand to sup_minus_dem DataFrame
            sup_minus_dem = sup_minus_dem.append({'water_supplier': d_utility, '2020': dif[0], '2025': dif[1], '2030': dif[2], '2035': dif[3]}, ignore_index = True)

Bottom 50 supply minus demand for year 2035. 

In [38]:
# Bottom 50 supply - demand for 2035
bot_50_sup_dem = sup_minus_dem.sort_values(by = ['2035'], ascending = False).nsmallest(50, '2035')

## 3.5) Top 50 Supply &/Minus Demand 

Determine which utilities are in top 50 for supply minus demand as well as supply and demand.

In [47]:
# Supply &/minus demand water suppliers
and_50 = both_top_50['water_supplier']
minus_50 = bot_50_sup_dem['water_supplier']

# Create all top 50 dataframe
all_top_50 = pd.DataFrame(columns = ['water_supplier', 'supply_avg_chg', '+/-', 'demand_avg_chg', '+/-', '2020', '2025', '2030', '2035'])

In [48]:
# For each top 50 supply - demand
for m_index, m_utility in minus_50.iteritems():
    
    # For each top 50 supply & minus
    for a_index, a_utility in and_50.iteritems():
        
        # If supply & demand == supply - minus
        if m_utility == a_utility:
            
            # If utility not already listed in all_top_50
            if a_utility not in all_top_50['water_supplier']:
                
                # Find supply & demand data
                sad = both_top_50.loc[a_index][1:5].to_numpy()

                # Find supply - demand data
                smd = bot_50_sup_dem.loc[m_index][1:5].to_numpy()

                # Add utility name, change, sign & difference to the all top 50 DataFrame
                all_top_50 = all_top_50.append({'water_supplier': a_utility, 'supply_avg_chg': sad[0], '+/-': sad[1], 'demand_avg_chg': sad[2], '+/-': sad[3], '2020': smd[0], '2025': smd[1], '2030': smd[2], '2035': smd[3]}, ignore_index = True)

## 4) Results

Top 50 supply changes.

In [42]:
top_50_sup

,water_supplier,supply_avg_chg,+/-
54,"Chowchilla, City of Water Department",61.601601,+
94,Hi Desert Water District,0.783197,-
10,Beaumont - Cherry Valley Water District,0.639903,-
190,South Feather Water and Power,0.611071,-
117,Marin Municipal Water District,0.603575,-
13,Benicia City Of,0.596912,-
144,Paradise Irrigation District,0.565220,-
160,Ripon City Of,0.529960,-
188,Shasta Lake City Of,0.500000,-
203,Ukiah City Of,0.500000,-


Top 50 demand changes.

In [43]:
top_50_dem

,water_supplier,demand_avg_chg,+/-
177,South Feather Water and Power,2.068829,+
61,Daly City,0.593120,+
58,Corona City Of,0.392462,+
105,Lincoln City Of,0.352347,-
142,Placer County Water Agency,0.345214,+
113,Madera City Of,0.325465,-
89,Groveland Community Services District,0.300606,-
149,Reedley City Of,0.300026,-
166,Sanger City Of,0.300022,-
107,Livingston City Of,0.299992,-


Utilities that appear in top 50 supplies & demands.

In [44]:
both_top_50

,water_supplier,supply_avg_chg,+/-,demand_avg_chg,+/-
0,Hi Desert Water District,0.783197,-,0.276684,-
1,South Feather Water and Power,0.611071,+,2.068829,+
2,Paradise Irrigation District,0.565220,-,0.266548,-
3,Ripon City Of,0.529960,-,0.199946,-
4,Carmichael Water District,0.437586,-,0.159969,-
5,Madera City Of,0.325465,-,0.325465,-
6,West Kern Water District,0.321277,-,0.159557,-
7,Triunfo Sanitation District/Oak Park Water Ser...,0.318807,-,0.182780,-
8,East Bay Municipal Utility District,0.312589,-,0.253524,-
9,Groveland Community Services District,0.300606,-,0.300606,-


Bottom 50 supply minus demand for 2035.

In [45]:
bot_50_sup_dem

,water_supplier,2020,2025,2030,2035
240,San Jose Water Company,-42884.638790,-22120.543700,-37041.476300,-54509.585900
80,East Bay Municipal Utility District,0.000000,-1120.144100,-13441.728800,-23523.025300
16,Beaumont - Cherry Valley Water District,-10517.000000,-11731.000000,-13451.000000,-14768.000000
303,Yuba City,-2163.565940,-4980.804980,-8246.101660,-12033.109260
124,Hayward City Of,-8715.641500,-9482.863460,-10587.663090,-11140.062900
239,San Jose City Of,-3796.000000,843.000000,-3782.000000,-9633.000000
42,California Water Service Company Los Altos/Sub...,-4318.000000,-2161.000000,-3532.000000,-5077.000000
230,San Buenaventura City Of (Ventura),-2645.000000,-3203.000000,-3653.000000,-4117.000000
44,California Water Service Company Mid Peninsula,-2436.000000,-2474.000000,-2620.000000,-2850.000000
212,Redwood City,-1700.000000,-2011.000000,-2137.000000,-2334.000000


Utilities that appear in top 50 supplies & demands as well as bottom 50 supply - demand.

In [49]:
all_top_50

,water_supplier,supply_avg_chg,+/-,demand_avg_chg,+/-,2020,2025,2030,2035
0,East Bay Municipal Utility District,0.312589,-,0.253524,-,0.0,-1120.1441,-13441.7288,-23523.0253
1,Triunfo Sanitation District/Oak Park Water Ser...,0.318807,-,0.182780,-,-30.0,-30.0000,-30.0000,-30.0000
2,Ripon City Of,0.529960,-,0.199946,-,0.0,0.0000,0.0000,0.0000
3,Pico Water District,0.270236,-,0.270236,-,0.0,0.0000,0.0000,0.0000
4,Sanger City Of,0.300022,-,0.300022,-,0.0,0.0000,0.0000,0.0000
